In [1]:
import socket

afqdn = socket.getfqdn() # your /etc/hosts should be set up to contain fully qualified domain name of the machine
afqdn

'nnlicv611.inn.intel.com'

In [2]:
!fx workspace create --prefix ~/workspace --template torch_cnn_histology
%cd ~/workspace

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-02 18:27:17.361264: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-02 18:27:17.361292: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Creating Workspace Directories
Creating Workspace Certifications
Creating Workspace Templates

New workspace directory structure:
workspace
├── code
│   ├── pthistology_inmemory.py
│   ├── pt_cnn.py
│   ├── keras_cnn.py
│   ├── histology_utils.py
│   ├── requirements.txt
│   ├── __pycache__
│   │   ├── pthistology_inmemory.cpython-36.pyc
│   │   ├── pt_cnn.cpython-36.pyc
│   │   ├── __init__.cpython-36.pyc
│   │   └── histology_utils.cpython-36.pyc
│   ├── mnist_utils.py
│   ├── __init__.py
│   └── tfmnist_inmemory.py
├── pl

In [3]:
!fx workspace certify

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-02 18:27:20.804965: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-02 18:27:20.804988: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Setting Up Certificate Authority...

1.  Create Root CA
1.1 Create Directories
1.2 Create Database
1.3 Create CA Request
1.4 Create CA Certificate
2.  Create Signing Certificate
2.1 Create Directories
2.2 Create Database
2.3 Create Signing Certificate CSR
2.4 Sign Signing Certificate CSR
3   Create Certificate Chain

Done.

 ✔️ OK


In [4]:
!fx aggregator generate-cert-request --fqdn $afqdn

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-02 18:27:23.328797: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-02 18:27:23.328824: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Creating AGGREGATOR certificate key pair with following settings: CN=nnlicv611.inn.intel.com, SAN=DNS:nnlicv611.inn.intel.com
  Moving AGGREGATOR certificate key pair to: cert/server

 ✔️ OK


In [5]:
!fx aggregator certify -s --fqdn $afqdn

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-02 18:27:25.620650: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-02 18:27:25.620673: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The CSR Hash for file agg_nnlicv611.inn.intel.com.csr =  05570d2fd47eed2bfa10d120b743adbbce6763899565374d8f6099eca1ce88a0

 Signing AGGREGATOR certificate key pair
  Moving AGGREGATOR certificate key pair to: cert/server

 ✔️ OK


In [6]:
!fx collaborator generate-cert-request -n one -d 1

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-02 18:27:27.826188: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-02 18:27:27.826212: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Creating COLLABORATOR certificate key pair with following settings: CN=one, SAN=DNS:one
  Moving COLLABORATOR certificate to: cert/col_one
Archive col_one_to_agg_cert_request.zip with certificate signing request created
This file should be sent to the certificate authority (typically hosted by the aggregator) for signing

 ✔️ OK


In [7]:
!fx collaborator certify -s -n one

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-02 18:27:30.104611: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-02 18:27:30.104638: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The CSR Hash for file col_one.csr =  42d65e339988ea9299f5cd16f19ea1a2611f3cc432e8d70e637542d474b17408

 Signing COLLABORATOR certificate key pair
  Moving COLLABORATOR certificate key pair to: cert/client

Registering one in plan/cols.yaml

 ✔️ OK


In [8]:
!fx collaborator generate-cert-request -n two -d 2

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-02 18:27:32.313255: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-02 18:27:32.313279: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Creating COLLABORATOR certificate key pair with following settings: CN=two, SAN=DNS:two
  Moving COLLABORATOR certificate to: cert/col_two
Archive col_two_to_agg_cert_request.zip with certificate signing request created
This file should be sent to the certificate authority (typically hosted by the aggregator) for signing

 ✔️ OK


In [9]:
!fx collaborator certify -s -n two

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-02 18:27:34.603731: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-02 18:27:34.603756: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
The CSR Hash for file col_two.csr =  520b848d9a319ef91187b40ed390e1cc9944154a9529fc86a1ff7bf32fbd9604

 Signing COLLABORATOR certificate key pair
  Moving COLLABORATOR certificate key pair to: cert/client

Registering two in plan/cols.yaml

 ✔️ OK


In [10]:
!fx plan initialize -a $afqdn

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-02 18:27:36.849734: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-02 18:27:36.849761: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[18:27:37] INFO     Loading DEFAULTS for section aggregator from file plan/defaults/aggregator.yaml.                                                  ]8;id=1606922857.8699079-370773;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:87
           INFO     Loading DEFAULTS for section assigner from file plan/defaults/assigner.yaml.                                                      ]8;id=1606922857.8717434-300529;file:///home/itrushkin/.virtualenvs/fledge/l

[18:27:39] INFO     Histology > X_train Shape : (2000, 3, 150, 150)                                                                       ]8;id=1606922859.486859-706496;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:128
           INFO     Histology > y_train Shape : (2000,)                                                                                   ]8;id=1606922859.4883358-245991;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:129
           INFO     Histology > Train Samples : 2000                                                                                      ]8;id=1606922859.4896574-479905;file:///home/itrushkin/workspace/code/histology_utils.py\histology_utils.py]8;;\:130
           INFO     Histology > Valid Samples : 500                                                                                       ]8;id=1606922859.4909852-488793;file:///home/itrushkin/workspace/code/histology_utils

In [11]:
!fx workspace export

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-02 18:27:43.074972: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-02 18:27:43.074998: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[18:27:44] INFO     FL-Plan hash is ab7a7d1dd524483741686fa58a3df558                                                                                 ]8;id=1606922864.1743047-808914;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:183
           INFO     plan.yaml frozen successfully                                                                                                     ]8;id=1606922864.1783352-969786;file:///home/itrushkin/.virtualenvs/fledge/l

In [ ]:
!fx aggregator start

Intel FLedge - Secure Federated Learning at the Edge™                           

2020-12-02 18:27:45.734542: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2020-12-02 18:27:45.734568: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[18:27:46] INFO     Loading DEFAULTS for section aggregator from file plan/defaults/aggregator.yaml.                                                  ]8;id=1606922866.7470603-734750;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:87
           INFO     Loading DEFAULTS for section assigner from file plan/defaults/assigner.yaml.                                                      ]8;id=1606922866.7488964-42667;file:///home/itrushkin/.virtualenvs/fledge/li

           INFO     Starting Aggregator gRPC Server                                                                                                ]8;id=1606922866.9001725-996669;file:///home/itrushkin/.virtualenvs/fledge/lib/python3.6/site-packages/fledge/transport/grpc/server.py\server.py]8;;\:145
